# Combining all CSVs

In [106]:
import pandas as pd

In [111]:
folder_path='output/'

In [117]:
files=[]
for foldername, subfolders, filenames in os.walk(folder_path):
    for filename in filenames:
        if 'new' in filename:
            files.append(os.path.join(foldername, filename))

In [176]:
files=['output/english\\04MCKCC03072017_new.csv',
 'output/english\\FARMER PRODUCER ORGANISATIONS_new.csv',
 'output/english\\FPO_Scheme_Guidelines_FINAL_English_new.csv',
 'output/english\\NFSM12102018_new.csv',
 'output/english\\RevisedPM-KISANOperationalGuidelines(English)_new.csv',
 'output/english\\Revised_Operational_Guidelines_new.csv',
 'output/english\\UGPs-Organic FarmingTechnologies and Strategies_new.csv',
 'output/hindi\\Natural_Farming_Kharif_Booklet_new_tabletranslated.csv',
 'output/hindi\\Pashupalan Margdarshika_new_tabletranslated.csv',
 'output/hindi\\RabiBookallpagesfullbook_2019_new_tabletranslated.csv']

In [177]:
combined_df=pd.DataFrame()
for file in files:
    df=pd.read_csv(file)
    combined_df=pd.concat([combined_df,df],ignore_index=True)

In [161]:
# df1=pd.read_csv(files[0])
# df2=pd.read_csv(files[1])
# combined_df=pd.concat([df1,df2],ignore_index=True)

In [178]:
combined_df['len_heading'] = combined_df['heading'].apply(lambda x: len(str(x).split()))

In [175]:
combined_df[combined_df.ContentWordCount == 1]

,Unnamed: 0,chunkId,content,heading,originalLanguageChunk,StartPage,EndPage,originalLanguageHeading,ContentWordCount,pdfName,contentType,image,summary,Unnamed: 0.1,len_heading
1820,128,212,"('Azadirectin', 0, 0.3273482322692871)",". दोमट या मटियार, कावर तथा मार :",| एजाडिरेक्टिन,NaN,NaN,". दोमट या मटियार, कावर तथा मार :",1,input/md/hindi/RabiBookallpagesfullbook_2019.pdf,table,NaN,NaN,NaN,8
1889,197,319,"('|समस्त', 0, 0.43642497062683105)",कं.सं. प्रजातियाँ अधिसूचना उत्पादकता पकने की व...,|समस्त,NaN,NaN,कं.सं. प्रजातियाँ अधिसूचना उत्पादकता पकने की व...,1,input/md/hindi/RabiBookallpagesfullbook_2019.pdf,table,NaN,NaN,NaN,15
1891,199,322,"('| All', 0, 0.37050461769104004)",कं.सं. प्रजातियाँ अधिसूचना उत्पादकता पकने की व...,| समस्त,NaN,NaN,कं.सं. प्रजातियाँ अधिसूचना उत्पादकता पकने की व...,1,input/md/hindi/RabiBookallpagesfullbook_2019.pdf,table,NaN,NaN,NaN,15
1906,214,348,"('Azadirectin', 0, 0.32911086082458496)",नियंत्रण के उपाय :,| एजाडिरेक्टिन,NaN,NaN,नियंत्रण के उपाय :,1,input/md/hindi/RabiBookallpagesfullbook_2019.pdf,table,NaN,NaN,NaN,4
2122,430,680,"('|तनों', 0, 0.3586595058441162)",बुआई का समय :,|तनों,NaN,NaN,बुआई का समय :,1,input/md/hindi/RabiBookallpagesfullbook_2019.pdf,table,NaN,NaN,NaN,4
2126,434,686,"('Infant', 0, 0.34640073776245117)",कॉर्न ।4. शिशु मक्का (बेबी कॉर्न) की खेती,| शिशु,NaN,NaN,कॉर्न ।4. शिशु मक्का (बेबी कॉर्न) की खेती,1,input/md/hindi/RabiBookallpagesfullbook_2019.pdf,table,NaN,NaN,NaN,8
2252,560,889,"('-5', 0, 0.32567501068115234)",+. भरसूर,|-5,NaN,NaN,+. भरसूर,1,input/md/hindi/RabiBookallpagesfullbook_2019.pdf,table,NaN,NaN,NaN,2
2301,609,965,"('the patient.', 0, 0.31939029693603516)",संस्तुत प्रजातियाँ,| रोगी,NaN,NaN,संस्तुत प्रजातियाँ,1,input/md/hindi/RabiBookallpagesfullbook_2019.pdf,table,NaN,NaN,NaN,2
2318,626,991,"('of this', 0, 0.33896684646606445)",उर्वरक :,| इसके,NaN,NaN,उर्वरक :,1,input/md/hindi/RabiBookallpagesfullbook_2019.pdf,table,NaN,NaN,NaN,2
2417,725,1138,"('...', 0, 0.29740476608276367)",बीज :,|...,NaN,NaN,बीज :,1,input/md/hindi/RabiBookallpagesfullbook_2019.pdf,table,NaN,NaN,NaN,2


In [184]:
combined_df.isna()

,Unnamed: 0,chunkId,content,heading,originalLanguageChunk,StartPage,EndPage,originalLanguageHeading,ContentWordCount,pdfName,contentType,image,summary,Unnamed: 0.1,len_heading
0,False,False,False,False,True,True,True,True,False,False,False,True,False,True,False
1,False,False,False,False,True,True,True,True,False,False,False,True,False,True,False
2,False,False,False,False,True,True,True,True,False,False,False,True,False,True,False
3,False,False,False,False,True,True,True,True,False,False,False,True,False,True,False
4,False,False,False,False,True,True,True,True,False,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3052,False,False,False,False,False,True,True,False,False,False,False,True,True,True,False
3053,False,False,False,False,False,True,True,False,False,False,False,True,True,True,False
3054,False,False,False,False,False,True,True,False,False,False,False,True,True,True,False
3055,False,False,False,False,False,True,True,False,False,False,False,True,True,True,False


# Initializing chunking

In [4]:
file_name='input/md/english/RevisedPM-KISANOperationalGuidelines(English).md'
md_file=open(file_name,'r',encoding='utf-8')
md=md_file.read()
chunks=md.split('\n')
chunks=[i.strip() for i in chunks]
md
# chunks=[chunk for chunk in chunks if chunk!='']
# for chunk in chunks:
#     print(chunk,end='\n*****\n')

'# Pradhan Mantri Kisan Samman Nidhi Scheme (Pm-Kisan Scheme)\n\n \n\n# Operational Guidelines\n\n## \n\n# (Revised As On 29.03.2020)\n\n \n\n## Ministry Of Agriculture & Farmers\' Welfare\n\n# Department Of Agriculture, Cooperation & Farmers\' Welfare (Dac&Fw)\n\n## Operational Guidelines\n\n\n## 1. Scheme\n\n With a view to provide income support to all landholding farmers\' families in the country, having cultivable land, the Central Government has implemented a Central Sector Scheme, namely, **"Pradhan Mantri Kisan Samman Nidhi (PM-KISAN)"**.  \n \n\n## 2. Objective And Benefits\n\n The scheme aims to supplement the financial needs of all landholding farmers\' families in procuring various inputs to ensure proper crop health and appropriate yields, commensurate with the anticipated farm income as well as for domestic needs. Under the Scheme an amount of Rs.6000/- per year is released by the Central Government online directly into the bank accounts of the eligible farmers under Dire

In [5]:
chunks

['# Pradhan Mantri Kisan Samman Nidhi Scheme (Pm-Kisan Scheme)',
 '',
 '',
 '',
 '# Operational Guidelines',
 '',
 '##',
 '',
 '# (Revised As On 29.03.2020)',
 '',
 '',
 '',
 "## Ministry Of Agriculture & Farmers' Welfare",
 '',
 "# Department Of Agriculture, Cooperation & Farmers' Welfare (Dac&Fw)",
 '',
 '## Operational Guidelines',
 '',
 '',
 '## 1. Scheme',
 '',
 'With a view to provide income support to all landholding farmers\' families in the country, having cultivable land, the Central Government has implemented a Central Sector Scheme, namely, **"Pradhan Mantri Kisan Samman Nidhi (PM-KISAN)"**.',
 '',
 '',
 '## 2. Objective And Benefits',
 '',
 "The scheme aims to supplement the financial needs of all landholding farmers' families in procuring various inputs to ensure proper crop health and appropriate yields, commensurate with the anticipated farm income as well as for domestic needs. Under the Scheme an amount of Rs.6000/- per year is released by the Central Government onlin

In [14]:
def table_parser(chunks,index):
    inside_table = False
    current_table = ''
    for i,chunk in enumerate(chunks):
        if inside_table:
            # Check if the current chunk is the end of the tablere
            if chunk.startswith('|') :
                inside_table = True
                current_table=current_table+'\n'+chunk
            else:

                return current_table,(index+i)
        elif chunk.startswith('|'):
            inside_table = True
            current_table=chunk
    #     elif chunk=='':
    #         continue
#         else:
#             print('post',(index+i))
#             return combined_chunks,(index+i)
    return 0


def para_parser(chunks,index):
    inside_para=False
    current_para=''
    temp=['#','## ','### ','#### ','##### ','###### ','-','|']
    for i,chunk in enumerate(chunks):
        if chunk=='':
            current_para=current_para.strip()
            return (current_para,(index+i))            
        elif inside_para:
            if chunk[0] not in temp:
                inside_para=True
                current_para=current_para+' '+chunk
            else:
                current_para=current_para.strip()
                # if len(current_para)==0:
                #     return(current_para,'flag')
                return (current_para,(index+i))            
        elif (chunk[0] not in temp) or chunk!='|':
            inside_para=True
            current_para=chunk
            
    return (current_para,(index+i))

def list_parser(chunks,index):
    inside_list=False
    current_list=''
    temp=['#','## ','### ','#### ','##### ','###### ','-','|']
    for i,chunk in enumerate(chunks):
        if inside_list:
            if chunk=='' or chunk.startswith('- ') or chunk.startswith('* '):
                return (current_list,(index+i))
            if chunk[0] not in temp:
                current_list=current_list+'\n'+chunk
        elif chunk.startswith('-') or chunk.startswith('* '):
            current_list=chunk
            inside_list=True

    return (current_list,(index+i))

# Extracting types for chunks and creating a JSON

In [17]:

i=0
combined_chunks=[]
temp=['#','## ','### ','#### ','##### ','###### ','-','*','|']
while i<len(chunks):
    
    if chunks[i]=='':
        combined_chunks.append({'type':'return','content':chunks[i]})
        i=i+1
    # elif chunks[i].startswith('|'):
    elif chunks[i].startswith('|') and len(chunks[i])!=1:
        chunk,i=table_parser(chunks[i:],i)
        combined_chunks.append({'type':'table','content':chunk})
        
    elif chunks[i].startswith('# ') or chunks[i].startswith('## ') or chunks[i].startswith('### ') or chunks[i].startswith('#### ') or chunks[i].startswith('##### '):
        combined_chunks.append({'type':'heading','content':chunks[i]})
        i=i+1
    
    elif chunks[i].startswith('- ') or chunks[i].startswith('* '):
        chunk,i=para_parser(chunks[i:],i)
        combined_chunks.append({'type':'list','content':chunk})
    
    # elif chunks[i][0] not in temp and (i+1)!=len(chunks):
    elif (chunks[i][0] not in temp and (i+1)!=len(chunks)) or chunks[i]=='|':
        chunk,i=para_parser(chunks[i:],i)
        
        # chunk=chunks[i].strip()
        if len(chunk)==0:
            i=i+1
            continue
        combined_chunks.append({'type':'para','content':chunk})
        # i=i+1
        
    else:
        i=i+1
    
combined_chunks = [chunk for chunk in combined_chunks if chunk['type'] != 'return']
# return combined_chunks

In [18]:
combined_chunks

[{'type': 'heading',
  'content': '# Pradhan Mantri Kisan Samman Nidhi Scheme (Pm-Kisan Scheme)'},
 {'type': 'heading', 'content': '# Operational Guidelines'},
 {'type': 'heading', 'content': '# (Revised As On 29.03.2020)'},
 {'type': 'heading',
  'content': "## Ministry Of Agriculture & Farmers' Welfare"},
 {'type': 'heading',
  'content': "# Department Of Agriculture, Cooperation & Farmers' Welfare (Dac&Fw)"},
 {'type': 'heading', 'content': '## Operational Guidelines'},
 {'type': 'heading', 'content': '## 1. Scheme'},
 {'type': 'para',
  'content': 'With a view to provide income support to all landholding farmers\' families in the country, having cultivable land, the Central Government has implemented a Central Sector Scheme, namely, **"Pradhan Mantri Kisan Samman Nidhi (PM-KISAN)"**.'},
 {'type': 'heading', 'content': '## 2. Objective And Benefits'},
 {'type': 'para',
  'content': "The scheme aims to supplement the financial needs of all landholding farmers' families in procuring v

In [94]:
len(combined_chunks)

200

In [95]:
inside_pair=False
paired_chunks=[]
current_pair={'type':'heading-para pair','content':{}}
for chunk in combined_chunks:
    if chunk['type']=='heading':
        inside_pair=True
        current_pair['content']['heading']=chunk['content']
    elif chunk['type']=='para':
        if inside_pair:
            current_pair['content']['para']=chunk['content']
            paired_chunks.append(current_pair)
            inside_pair=False
            current_pair={'type':'heading-para pair','content':{}}
        else:
            paired_chunks.append(chunk)
    else:
        paired_chunks.append(chunk)

In [ ]:
# def count_level(content):
#     level
#     if content.startswith('# '):
#         level=1
#     elif content.startswith('## '):
#         level=2
#     elif content.startswith('### '):
#         level=3
#     elif content.startswith('#### '):
#         level=4
#     elif content.startswith('##### '):
#         level=5
#     elif content.startswith('###### '):
#         level=6


#     return level

# Creating heading heirarchy

In [19]:
def assign_ids_and_parents(chunks):
    id_counter = {'1': 0, '2': 0, '3': 0,'4':0,'5':0}
    parent_stack = []

    for chunk in chunks:
        if chunk['type'] == 'heading':
            level = chunk['content'].count('#')
#             level=count_level(chunk['content'])
#             print('hehe',level)
            id_counter[str(level)] += 1
            chunk['id'] = f'{level}{id_counter[str(level)]}'

            if parent_stack==[] or level==1:
                parent_stack = [chunk['id']]
            else:
                buff=int(str(parent_stack[0])[0])-1
                while len(parent_stack)>=(level-buff):
                    parent_stack.pop()
                parent_stack.append(chunk['id'])
            chunk['parents'] = parent_stack[:-1]
#             print(parent_stack)

        else:
            chunk['parents'] = parent_stack.copy() if parent_stack else []

    return chunks
final_chunks=assign_ids_and_parents(combined_chunks)
final_chunks

[{'type': 'heading',
  'content': '# Pradhan Mantri Kisan Samman Nidhi Scheme (Pm-Kisan Scheme)',
  'id': '11',
  'parents': []},
 {'type': 'heading',
  'content': '# Operational Guidelines',
  'id': '12',
  'parents': []},
 {'type': 'heading',
  'content': '# (Revised As On 29.03.2020)',
  'id': '13',
  'parents': []},
 {'type': 'heading',
  'content': "## Ministry Of Agriculture & Farmers' Welfare",
  'id': '21',
  'parents': ['13']},
 {'type': 'heading',
  'content': "# Department Of Agriculture, Cooperation & Farmers' Welfare (Dac&Fw)",
  'id': '14',
  'parents': []},
 {'type': 'heading',
  'content': '## Operational Guidelines',
  'id': '22',
  'parents': ['14']},
 {'type': 'heading', 'content': '## 1. Scheme', 'id': '23', 'parents': ['14']},
 {'type': 'para',
  'content': 'With a view to provide income support to all landholding farmers\' families in the country, having cultivable land, the Central Government has implemented a Central Sector Scheme, namely, **"Pradhan Mantri Kisa

In [55]:
final_chunks

[{'type': 'heading',
  'content': '# Pradhan Mantri Kisan Samman Nidhi Scheme (Pm-Kisan Scheme)',
  'id': '11',
  'parents': []},
 {'type': 'heading',
  'content': '# Operational Guidelines',
  'id': '12',
  'parents': []},
 {'type': 'heading',
  'content': '# (Revised As On 29.03.2020)',
  'id': '13',
  'parents': []},
 {'type': 'heading',
  'content': "## Ministry Of Agriculture & Farmers' Welfare",
  'id': '21',
  'parents': ['13']},
 {'type': 'heading',
  'content': "# Department Of Agriculture, Cooperation & Farmers' Welfare (Dac&Fw)",
  'id': '14',
  'parents': []},
 {'type': 'heading',
  'content': '## Operational Guidelines',
  'id': '22',
  'parents': ['14']},
 {'type': 'heading', 'content': '## 1. Scheme', 'id': '23', 'parents': ['14']},
 {'type': 'para',
  'content': 'With a view to provide income support to all landholding farmers\' families in the country, having cultivable land, the Central Government has implemented a Central Sector Scheme, namely, **"Pradhan Mantri Kisa

# Creating CSV

In [21]:
headings={}
for chunk in final_chunks:
    if chunk['type']=='heading':
        headings[chunk['id']]=chunk['content']

In [29]:
def get_subheading(chunks,index):
  subheadings=''
  for heading in chunks[index]['parents']:
    subheadings+=(' '.join(headings[heading].split()[1:]))+','
  return subheadings[:-1]

import csv
#CSV columns: chunkId,content,StartPage,EndPage,heading,ContentWordCount,pdfName,contentType,image,summary
def export_csv(final_chunks):

  with open('trial RevisedPM-KISANOperationalGuidelines(English).csv','w',newline='') as file:
    csv_writer = csv.writer(file)
    data=['chunkId','content','StartPage','EndPage','heading','ContentWordCount','pdfName','contentType','image','summary']
    csv_writer.writerow(data)

  for i,chunk in enumerate(final_chunks):
    chunkId=i+1
    content=chunk['content']
    heading=get_subheading(final_chunks,i)
    contentType=chunk['type']
    ContentWordCount=len(content.split()) if contentType!='table' else len(content.split('|'))
    pdfName=file_name.split('.')[0]+'.pdf'
    data=[chunkId,content,'','',heading,ContentWordCount,pdfName,contentType,'','']
    with open('trial RevisedPM-KISANOperationalGuidelines(English).csv','a',encoding='utf-8',newline='') as file:
      csv_writer = csv.writer(file)
      csv_writer.writerow(data)




In [23]:
original_string = "This is a simple string."

# Split the string into words, add a space after each word, and join them
modified_string = ' '.join(original_string.split()[1:])

# Display the modified string
print(modified_string)

is a simple string.


In [30]:
export_csv(final_chunks)

# Translating

In [12]:
import requests
import json
import pandas as pd
import time

def translate_text_bhashini(input_text):
    url = "https://dhruva-api.bhashini.gov.in/services/inference/pipeline"

    payload = json.dumps({
        "pipelineTasks": [
            {
                "taskType": "translation",
                "config": {
                    "language": {
                        "sourceLanguage": "hi",
                        "targetLanguage": "en"
                    },
                    "serviceId": "ai4bharat/indictrans-v2-all-gpu--t4"
                }
            }
        ],
        "inputData": {
            "input": [
                {
                    "source": input_text
                }
            ]
        }
    })
    headers = {
        'Accept': '*/*',
        'User-Agent': 'Thunder Client (https://www.thunderclient.com)',
        'Authorization': 'sLAFJehUCZQ72NIM4nDZNCya7TQVzittLgJEU0vIf-69rp0gFUcGu5sjwAaOSUfa',
        'Content-Type': 'application/json'
    }

    retries = 0
    max_retries = 5
    start_time = time.time()
    while retries < max_retries:
        try:
            response = requests.post(url, headers=headers, data=payload)
            # Check if the request was successful
            if response.status_code == 200:
                translated_output = json.loads(response.text)['pipelineResponse'][0]['output'][0]['target']
                end_time = time.time()
                return translated_output, retries, end_time - start_time
            else:
                print(f"Request failed with status code {response.status_code}. Retrying...")
                retries += 1
        except Exception as e:
            print(f"An error occurred: {e}. Retrying...")
            retries += 1

    end_time = time.time()
    return input_text

In [33]:
def modify_content(row):
    if row['contentType']!='table':
        return row['content']
    else:
        return translate_text_bhashini(row['content'])

In [44]:
import pandas as pd
data=pd.read_csv('output/hindi/Pashupalan Margdarshika_new.csv')

In [45]:
data['content'] = data.apply(modify_content, axis=1)

In [47]:
data.to_csv('output/hindi/Pashupalan Margdarshika_new_tabletranslated.csv')

In [46]:
data.tail(25)

,Unnamed: 0,chunkId,content,heading,originalLanguageChunk,StartPage,EndPage,originalLanguageHeading,ContentWordCount,pdfName,contentType,image,summary
97,97,71,"Turmeric, 20 gm Ajwain powder 250 gm The jagge...",Pemphigic techniques for animal health,"प्राचीन काल से मनुष्य अपने भोजन, स्वास्थ्य एवं...",NaN,NaN,पशु स्वास्थ्य हेतु पाएम्पएिक तकनीकियाँ,200,input/md/hindi/Pashupalan Margdarshika.pdf,para,NaN,The traditional method for treating indigestio...
98,98,71,"00 gm. Black salt, 30 gm asafoetida, 400 ml tu...",Pemphigic techniques for animal health,"प्राचीन काल से मनुष्य अपने भोजन, स्वास्थ्य एवं...",NaN,NaN,पशु स्वास्थ्य हेतु पाएम्पएिक तकनीकियाँ,199,input/md/hindi/Pashupalan Margdarshika.pdf,para,NaN,The given content provides various techniques ...
99,99,71,linseed oil to small animal and 200 ml. L. Giv...,Pemphigic techniques for animal health,"प्राचीन काल से मनुष्य अपने भोजन, स्वास्थ्य एवं...",NaN,NaN,पशु स्वास्थ्य हेतु पाएम्पएिक तकनीकियाँ,155,input/md/hindi/Pashupalan Margdarshika.pdf,para,NaN,"Linseed oil is recommended for small animals, ..."
100,100,71,Health management in chickens: Farmer brothers...,Pemphigic techniques for animal health,"प्राचीन काल से मनुष्य अपने भोजन, स्वास्थ्य एवं...",NaN,NaN,पशु स्वास्थ्य हेतु पाएम्पएिक तकनीकियाँ,60,input/md/hindi/Pashupalan Margdarshika.pdf,para,NaN,The farmer brothers use various techniques for...
101,101,72,Q. What are the measures to prevent sudden dro...,Questions and Answers for Animal Husbandry,प्र0- दुधारू पशुओं में एकाएक दुग्ध उत्पादन कयो...,NaN,NaN,पशुपालन जन्य प्रश्न एवं उत्तर,194,input/md/hindi/Pashupalan Margdarshika.pdf,para,NaN,The measures to prevent sudden drop in milk pr...
102,102,72,"Its main medications, such as Varenil, Triquin...",Questions and Answers for Animal Husbandry,प्र0- दुधारू पशुओं में एकाएक दुग्ध उत्पादन कयो...,NaN,NaN,पशुपालन जन्य प्रश्न एवं उत्तर,78,input/md/hindi/Pashupalan Margdarshika.pdf,para,NaN,The main medications for animal husbandry shou...
103,103,73,Q. Where does the blood come from all around t...,Questions and Answers for Animal Husbandry,प्र0- गाय के ब्याने के बाद उसके चारो थनों से ख...,NaN,NaN,पशुपालन जन्य प्रश्न एवं उत्तर,195,input/md/hindi/Pashupalan Margdarshika.pdf,para,NaN,The bleeding from all four extremities of a sl...
104,104,73,The urea content is 4 kg / 400 kg. G. Should n...,Questions and Answers for Animal Husbandry,प्र0- गाय के ब्याने के बाद उसके चारो थनों से ख...,NaN,NaN,पशुपालन जन्य प्रश्न एवं उत्तर,195,input/md/hindi/Pashupalan Margdarshika.pdf,para,NaN,The urea content should not exceed 4 kg / 400 ...
105,105,73,A: Your animal has symptoms of milk fever. Mil...,Questions and Answers for Animal Husbandry,प्र0- गाय के ब्याने के बाद उसके चारो थनों से ख...,NaN,NaN,पशुपालन जन्य प्रश्न एवं उत्तर,193,input/md/hindi/Pashupalan Margdarshika.pdf,para,NaN,Animals with symptoms of milk fever should be ...
106,106,73,There is difficulty in breathing and the anima...,Questions and Answers for Animal Husbandry,प्र0- गाय के ब्याने के बाद उसके चारो थनों से ख...,NaN,NaN,पशुपालन जन्य प्रश्न एवं उत्तर,199,input/md/hindi/Pashupalan Margdarshika.pdf,para,NaN,"Animals experiencing difficulty in breathing, ..."


In [ ]:
a

In [ ]:
import os
input_folder='output\\hindi'
all_files=[]
for folder, subfolders, files in os.walk(input_folder):
    full_file_paths = [os.path.join(folder, file) for file in files]
    all_files.extend(full_file_paths)
all_files

In [ ]:
import pandas as pd
df=pd.read_csv(all_files[0])
newdf=pd.read_csv('output.csv')

In [ ]:
newdf

In [ ]:
df = df.rename(columns={'content': 'originalLanguageChunk','heading':'originalLanguageHeading'})

In [ ]:
print('translating')
df['content'] = df.apply(lambda row: translate_text_bhashini(row['originalLanguageChunk'])[0] if row['contentType'] != 'table' else row['originalLanguageChunk'], axis=1)
df['heading'] = df.apply(lambda row: translate_text_bhashini(row['originalLanguageHeading'])[0] if row['contentType'] != 'table' else row['originalLanguageHeading'], axis=1)
print('\rtranslating done')
df=df[]

In [ ]:
df=df[['chunkId','content','heading','originalLanguageChunk','StartPage','EndPage','originalLanguageHeading','ContentWordCount','pdfName','contentType','image','summary']]

In [18]:
df

,Unnamed: 0,chunkId,content,heading,originalLanguageChunk,StartPage,EndPage,originalLanguageHeading,ContentWordCount,pdfName,contentType,image,summary,len_heading
0,0,1,"Department of Agriculture, Cooperation and Far...",Part –I (B) National Food Security Mission (Nf...,NaN,NaN,NaN,NaN,175,input/md/english/NFSM12102018.pdf,para,NaN,This document outlines the operational guideli...,20
1,1,2,| S.No. ...,Re-Vamped National Food Security Mission (Nfsm...,NaN,NaN,NaN,NaN,384,input/md/english/NFSM12102018.pdf,table,NaN,The document outlines the operational guidelin...,13
2,2,4,| S.No. ...,Re-Vamped National Food Security Mission (Nfsm...,NaN,NaN,NaN,NaN,311,input/md/english/NFSM12102018.pdf,table,NaN,The document contains the table of contents fo...,13
3,3,5,| S.No. ...,Re-Vamped National Food Security Mission (Nfsm...,NaN,NaN,NaN,NaN,222,input/md/english/NFSM12102018.pdf,table,NaN,The document contains the table of contents fo...,13
4,4,6,| S.No. ...,Re-Vamped National Food Security Mission (Nfsm...,NaN,NaN,NaN,NaN,55,input/md/english/NFSM12102018.pdf,table,NaN,This section provides the table of contents fo...,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,601,875,Target Achievement Financial Physica Tota...,Part-Ii National Food Security Mission-Commerc...,NaN,NaN,NaN,NaN,111,input/md/english/NFSM12102018.pdf,para,NaN,This form is for the submission of monthly and...,17
602,602,786,1. Distribution of Seeds: 1.1 Cereal crops: ...,National Food Security Mission (Nfsm) (Seed Vi...,NaN,NaN,NaN,NaN,195,input/md/english/NFSM12102018.pdf,para,NaN,Financial assistance is provided for the distr...,10
603,603,786,capacityupto maximum of Rs.1000 @25% for Gener...,National Food Security Mission (Nfsm) (Seed Vi...,NaN,NaN,NaN,NaN,58,input/md/english/NFSM12102018.pdf,para,NaN,The Seed Village Programme under the National ...,10
604,604,787,The pattern of assistance available for Seed ...,National Food Security Mission (Nfsm) (Seed Vi...,NaN,NaN,NaN,NaN,196,input/md/english/NFSM12102018.pdf,para,NaN,The Seed Village Programme under the National ...,17


# Utils

In [43]:
import pandas as pd
import re

def split_chunks(df, upper_limit=200):
    new_rows = []
    count=0
    for index, row in df.iterrows():
        
    
        word_count = row['ContentWordCount']
        content = row['content']
        
        if int(word_count) > upper_limit and (row['contentType']=='para' or row['contentType']=='list'):
            sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|;|\?)\s', content)
            
            
            chunks = []
            current_chunk = []
            for sentence in sentences:
                # print(sentence)
                
                temp=' '.join(current_chunk + [sentence])
                if len(temp.split()) <= upper_limit:
                    current_chunk.append(sentence)
                    # print(current_chunk)
                    
                else:
                    chunks.append(' '.join(current_chunk))
                    current_chunk = [sentence]
            
            if current_chunk:
                chunks.append(' '.join(current_chunk))
    
            
            
            for chunk in chunks:
                count+=1
                new_row = {
                    'chunkId': index,  
                    'content': chunk,
                    'heading': row['heading'],
                    'originalLanguageChunk': row['originalLanguageChunk'],
                    'StartPage': row['StartPage'],
                    'EndPage': row['EndPage'],
                    'originalLanguageHeading': row['originalLanguageHeading'],
                    'ContentWordCount': len(chunk.split()),
                    'pdfName': row['pdfName'],
                    'contentType': row['contentType'],
                    'image': row['image'],
                    'summary': row['summary']
                }
                new_rows.append(new_row)
        else:
           
            count+=1
            new_rows.append({
                    'chunkId': count,
                    'content': content,
                    'heading': row['heading'],
                    'originalLanguageChunk': row['originalLanguageChunk'],
                    'StartPage': row['StartPage'],
                    'EndPage': row['EndPage'],            
                    'originalLanguageHeading': row['originalLanguageHeading'],
                    'ContentWordCount': word_count,
                    'pdfName': row['pdfName'],
                    'contentType': row['contentType'],
                    'image': row['image'],
                    'summary': row['summary']
            })


    new_df = pd.DataFrame(new_rows)
    return new_df

def merge_small_chunks(df,lowerLimit=20):
    df['status']='visited'
    rows = df.to_dict(orient='records')
    size=len(rows)
    i=0
    lowerLimit=40
    while i<size:
        
        row=rows[i]
        if row['contentType']=='table':
            i+=1
            continue
        if row['status']=='unvisited':
            i+=1
            continue
        # print(row['chunkId'])
        
        stat=(i/size)*100
        print(stat,'% done',end='\r')
        prev_row=rows[i-1] if i>0 else None
        next_row=rows[i+1] if i<size-1 else None
        
        # if type(str(next_row))=='pandas.core.series.Series':

        # merged_rows = []
        # current_row = None
        temp=1
        while row['ContentWordCount']<lowerLimit:
            print('here')
            
            if isinstance(next_row, dict):
                
                same_type=(row['contentType']==next_row['contentType'])
                same_heading=row['heading']==next_row['heading']
                valid_word_count=row['ContentWordCount']+next_row['ContentWordCount']<200 
            else:
                same_type=same_heading=valid_word_count=False            
            if isinstance(prev_row, dict):
                psame_type=(row['contentType']==prev_row['contentType'])
                psame_heading=(row['heading']==prev_row['heading'])
                pvalid_word_count=row['ContentWordCount']+prev_row['ContentWordCount']<200
            else:
                psame_type=psame_heading=pvalid_word_count=False
        
            if  same_type and same_heading and valid_word_count:
                print('here')
                row['content']+=next_row['content']
                row['ContentWordCount']+=next_row['ContentWordCount']
                next_row['status']='unvisited'
                
            # elif same_type and not same_heading:
            elif psame_type and psame_heading and pvalid_word_count:
                row['content']+=prev_row['content']
                row['ContentWordCount']+=prev_row['ContentWordCount']
                prev_row['status']='unvisited'

            elif valid_word_count and same_heading:
                row['content']+=str(next_row['content'])
                row['ContentWordCount']+=next_row['ContentWordCount']
                next_row['status']='unvisited'

            elif pvalid_word_count and psame_heading:
                row['content']+=prev_row['content']
                row['ContentWordCount']+=prev_row['ContentWordCount']
                prev_row['status']='unvisited'

            elif valid_word_count:
                row['content']+=next_row['content']
                row['ContentWordCount']+=next_row['ContentWordCount']
                next_row['status']='unvisited'
                merged=str(row['heading'])+','+str(next_row['heading'])
                merged=list(set(merged.split(',')))
                
                row['heading']=','.join(merged)
            
            elif pvalid_word_count:
                row['content']+=prev_row['content']
                row['ContentWordCount']+=prev_row['ContentWordCount']
                prev_row['status']='unvisited'
                merged=str(row['heading'])+','+str(prev_row['heading'])
                merged=list(set(merged.split(',')))
                # row['heading']=','.join(list(set([(row['heading']+','+prev_row['heading'])].split(','))))

            else:
                print('reached')
                temp+=1
                
                prev_row=rows[i-temp] if i>temp-1 else None
                # print(prev_row)
                # break
                next_row=rows[i+temp] if i<size-temp else None
                # print(prev_row)
                
        # break
                # temp+=1
                # if i>temp-1:
                #      prev_row=rows[i-temp]
                # else:
                #     prev_row=None
                # next_row=rows[i+temp] if i<size-temp else None

        i+=1

    columns = ['chunkId', 'content', 'heading', 'originalLanguageChunk', 'StartPage', 'EndPage', 'originalLanguageHeading', 'ContentWordCount', 'pdfName', 'contentType', 'image', 'summary', 'status']

    new_df = pd.DataFrame(rows, columns=columns)
    new_df = new_df[new_df['status']=='visited']
    new_df=new_df.drop(columns=['status'])
    return new_df

# Splitting bigger chunks

In [33]:
import re
import pandas as pd

In [34]:
df=pd.read_csv('trial RevisedPM-KISANOperationalGuidelines(English).csv')

In [35]:
df

,chunkId,content,StartPage,EndPage,heading,ContentWordCount,pdfName,contentType,image,summary
0,1,# Pradhan Mantri Kisan Samman Nidhi Scheme (Pm...,NaN,NaN,NaN,9,input/md/english/RevisedPM-KISANOperationalGui...,heading,NaN,NaN
1,2,# Operational Guidelines,NaN,NaN,NaN,3,input/md/english/RevisedPM-KISANOperationalGui...,heading,NaN,NaN
2,3,# (Revised As On 29.03.2020),NaN,NaN,NaN,5,input/md/english/RevisedPM-KISANOperationalGui...,heading,NaN,NaN
3,4,## Ministry Of Agriculture & Farmers' Welfare,NaN,NaN,(Revised As On 29.03.2020),7,input/md/english/RevisedPM-KISANOperationalGui...,heading,NaN,NaN
4,5,"# Department Of Agriculture, Cooperation & Far...",NaN,NaN,NaN,9,input/md/english/RevisedPM-KISANOperationalGui...,heading,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
57,58,viii. The sponsoring bank is to transmit the f...,NaN,NaN,"Department Of Agriculture, Cooperation & Farme...",20,input/md/english/RevisedPM-KISANOperationalGui...,para,NaN,NaN
58,59,## B. Union Government:,NaN,NaN,"Department Of Agriculture, Cooperation & Farme...",4,input/md/english/RevisedPM-KISANOperationalGui...,heading,NaN,NaN
59,60,I. The Programme Division in the DAC&FW is to ...,NaN,NaN,"Department Of Agriculture, Cooperation & Farme...",218,input/md/english/RevisedPM-KISANOperationalGui...,para,NaN,NaN
60,61,Validity of the list of beneficiaries,NaN,NaN,"Department Of Agriculture, Cooperation & Farme...",6,input/md/english/RevisedPM-KISANOperationalGui...,para,NaN,NaN


In [40]:
df.to_csv("trial RevisedPM-KISANOperationalGuidelines(English).csv")

In [41]:
df['originalLanguageChunk']=''
df['originalLanguageHeading']=''
df=df[['chunkId','content','heading','originalLanguageChunk','StartPage','EndPage','originalLanguageHeading','ContentWordCount','pdfName','contentType','image','summary']]

In [39]:
df_filtered = df[df['contentType'] != 'heading']
df_filtered = df_filtered[df_filtered['ContentWordCount']!=0]
df=df_filtered

In [53]:
splitted=split_chunks(combined,upper_limit=200)

In [54]:
splitted.to_csv('trial RevisedPM-KISANOperationalGuidelines(English).csv')

In [52]:
combined=merge_small_chunks(df,lowerLimit=20)

here% done
here5454545454546 % done
here
here0909090909092 % done
here
here
here
here09090909090908 % done
here
here27272727272727 % done
here
here % done727273 % donee
here4545454545454 % done
here
here3636363636363 % done
here
here
here
here8181818181817 % done
here
here7272727272727 % done
here1818181818183 % done
here
here6363636363636 % done
here
here090909090909 % done
here5454545454545 % done
here


C:\Users\rachi\AppData\Local\Temp\ipykernel_8928\2143408513.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['status']='visited'


In [51]:
combined.to_csv('trial revised operational giudelines.csv')

In [65]:
combined

,chunkId,content,heading,originalLanguageChunk,StartPage,EndPage,originalLanguageHeading,ContentWordCount,pdfName,contentType,image,summary
0,1,Pradhan Mantri Fasal Bima Yojana(PMFBY)(PMFBY)...,Operational Guidelines,,NaN,NaN,,40,input/md/english/Revised_Operational_Guideline...,para,NaN,NaN
2,3,"(Revised)Department of Agriculture, Cooperatio...",Operational Guidelines,,NaN,NaN,,52,input/md/english/Revised_Operational_Guideline...,para,NaN,NaN
4,5,| Sl. No Section ...,Index,,NaN,NaN,,136,input/md/english/Revised_Operational_Guideline...,table,NaN,NaN
5,6,| Table No. ...,Table Index,,NaN,NaN,,127,input/md/english/Revised_Operational_Guideline...,table,NaN,NaN
6,7,| AIC | Agricultural Insurance Company of ...,Abbreviations,,NaN,NaN,,136,input/md/english/Revised_Operational_Guideline...,table,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2466,2467,Organizing State level training/capacity build...,Other Parameters,,NaN,NaN,,40,input/md/english/Revised_Operational_Guideline...,para,NaN,NaN
2467,2468,"- Banks, CSCs and Departmental workers- For ...",Other Parameters,,NaN,NaN,,79,input/md/english/Revised_Operational_Guideline...,list,NaN,NaN
2469,2470,| |\r\n|...,Other Parameters,,NaN,NaN,,493,input/md/english/Revised_Operational_Guideline...,table,NaN,NaN
2470,2471,| Threshold limit ...,Performance Severity Of States,,NaN,NaN,,31,input/md/english/Revised_Operational_Guideline...,table,NaN,NaN


In [ ]:
new_rows = []
count=0
for index, row in df.iterrows():
    
    
    word_count = row['ContentWordCount']
    content = row['content']
    
    # Check if ContentWordCount is greater than 200
    if int(word_count) > 200 and row['contentType']=='para':
        # Split the content into chunks of <= 200 words with complete sentences
        sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', content)
        
        
        chunks = []
        current_chunk = []
        for sentence in sentences:
            # print(sentence)
            
            temp=' '.join(current_chunk + [sentence])
            if len(temp.split()) <= 200:
                current_chunk.append(sentence)
                # print(current_chunk)
                
            else:
                chunks.append(' '.join(current_chunk))
                current_chunk = [sentence]
        
        if current_chunk:
            chunks.append(' '.join(current_chunk))

        # Create new rows for each chunk
        
        for chunk in chunks:
            count+=1
            new_row = {
                'chunkId': index,  # You may need to assign a new chunkId
                'content': chunk,
                'heading': row['heading'],
                'originalLanguageChunk': row['originalLanguageChunk'],
                'StartPage': row['StartPage'],
                'EndPage': row['EndPage'],
                'originalLanguageHeading': row['originalLanguageHeading'],
                'ContentWordCount': len(chunk.split()),
                'pdfName': row['pdfName'],
                'contentType': row['contentType'],
                'image': row['image'],
                'summary': row['summary']
            }
            new_rows.append(new_row)
    else:
        # If ContentWordCount is <= 200, keep the row as it is
        count+=1
        new_rows.append({
            'chunkId': count,
            'content': content,
            'heading': row['heading'],
            'originalLanguageChunk': row['originalLanguageChunk'],
            'StartPage': row['StartPage'],
            'EndPage': row['EndPage'],            
            'originalLanguageHeading': row['originalLanguageHeading'],
            'ContentWordCount': word_count,
            'pdfName': row['pdfName'],
            'contentType': row['contentType'],
            'image': row['image'],
            'summary': row['summary']
        })

# Create a new DataFrame with the updated rows
new_df = pd.DataFrame(new_rows)


In [ ]:
new_df

In [ ]:
new_d

In [ ]:
sentence="and Rural Development National Bank for Agriculture Mumbai  2015 Title                               :                                  Farmer Producer Organisations - Frequently Asked Questions (FAQs)   Written and Published by                               :                                  Farm Sector Policy Department & Farm Sector                                   Development Department, NABARD Head Office, Mumbai  Date of Publishing                               :                                  March 2015 Design & Printing                               :                                  M/s Image Impression Contact                               :                                  Plot No C-24, 'G' Block, Bandra Kurla Complex,                                  Bandra East, Mumbai - 400051."

In [ ]:
a=' '.join(current_chunk + [sentence])

In [ ]:
len(a)

# 2.0

In [ ]:
import pandas as pd
import re
def split_chunks(df,upper_limit=180):
    new_rows = []
    count=0
    for index, row in df.iterrows():
        
    
        word_count = row['ContentWordCount']
        content = row['content']
        
        if int(word_count) > upper_limit and row['contentType']=='para':
            sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', content)
            
            
            chunks = []
            current_chunk = []
            for sentence in sentences:
                # print(sentence)
                
                temp=' '.join(current_chunk + [sentence])
                if len(temp.split()) <= upper_limit:
                    current_chunk.append(sentence)
                    # print(current_chunk)
                    
                else:
                    chunks.append(' '.join(current_chunk))
                    current_chunk = [sentence]
            
            if current_chunk:
                chunks.append(' '.join(current_chunk))
    
            
            
            for chunk in chunks:
                count+=1
                new_row = {
                    'chunkId': index,  
                    'content': chunk,
                    'heading': row['heading'],
                    'originalLanguageChunk': row['originalLanguageChunk'],
                    'StartPage': row['StartPage'],
                    'EndPage': row['EndPage'],
                    'originalLanguageHeading': row['originalLanguageHeading'],
                    'ContentWordCount': len(chunk.split()),
                    'pdfName': row['pdfName'],
                    'contentType': row['contentType'],
                    'image': row['image'],
                    'summary': row['summary']
                }
                new_rows.append(new_row)
        else:
           
            count+=1
            new_rows.append({
                    'chunkId': count,
                    'content': content,
                    'heading': row['heading'],
                    'originalLanguageChunk': row['originalLanguageChunk'],
                    'StartPage': row['StartPage'],
                    'EndPage': row['EndPage'],            
                    'originalLanguageHeading': row['originalLanguageHeading'],
                    'ContentWordCount': word_count,
                    'pdfName': row['pdfName'],
                    'contentType': row['contentType'],
                    'image': row['image'],
                    'summary': row['summary']
            })


    new_df = pd.DataFrame(new_rows)
    return new_df

In [ ]:
df=pd.read_csv('output/english\\NFSM12102018.csv')

In [ ]:
new_df=split_chunks(df)

In [ ]:
new_df

# Joining Rows

In [115]:
import pandas as pd
df=pd.read_csv('output/english/Revised_Operational_Guidelines_new.csv')
df['status']='visited'

In [116]:
rows = df.to_dict(orient='records')

In [119]:
print('%')

%


In [118]:
rows = df.to_dict(orient='records')
size=len(rows)
i=0
lowerLimit=40
while i<size:
    
    row=rows[i]
    if row['status']=='unvisited':
        i+=1
        continue
    # print(row['chunkId'])
    print(i)
    prev_row=rows[i-1] if i>0 else None
    next_row=rows[i+1] if i<size-1 else None
    
    # if type(str(next_row))=='pandas.core.series.Series':

    # merged_rows = []
    # current_row = None
    temp=1
    while row['ContentWordCount']<lowerLimit:
        print('here')
        
        if isinstance(next_row, dict):
            
            same_type=(row['contentType']==next_row['contentType'])
            same_heading=row['heading']==next_row['heading']
            valid_word_count=row['ContentWordCount']+next_row['ContentWordCount']<200 
        else:
            same_type=same_heading=valid_word_count=False            
        if isinstance(prev_row, dict):
            psame_type=(row['contentType']==prev_row['contentType'])
            psame_heading=(row['heading']==prev_row['heading'])
            pvalid_word_count=row['ContentWordCount']+prev_row['ContentWordCount']<200
        else:
            psame_type=psame_heading=pvalid_word_count=False
       
        if  same_type and same_heading and valid_word_count:
            print('here')
            row['content']+=next_row['content']
            row['ContentWordCount']+=next_row['ContentWordCount']
            next_row['status']='unvisited'
            
        # elif same_type and not same_heading:
        elif psame_type and psame_heading and pvalid_word_count:
            row['content']+=prev_row['content']
            row['ContentWordCount']+=prev_row['ContentWordCount']
            prev_row['status']='unvisited'

        elif valid_word_count and same_heading:
            row['content']+=next_row['content']
            row['ContentWordCount']+=next_row['ContentWordCount']
            next_row['status']='unvisited'

        elif pvalid_word_count and psame_heading:
            row['content']+=prev_row['content']
            row['ContentWordCount']+=prev_row['ContentWordCount']
            prev_row['status']='unvisited'

        elif valid_word_count:
            row['content']+=(next_row['content'])
            row['ContentWordCount']+=next_row['ContentWordCount']
            next_row['status']='unvisited'
            merged=str(row['heading'])+','+str(next_row['heading'])
            merged=list(set(merged.split(',')))
            
            row['heading']=','.join(merged)
        
        elif pvalid_word_count:
            row['content']+=prev_row['content']
            row['ContentWordCount']+=prev_row['ContentWordCount']
            prev_row['status']='unvisited'
            merged=str(row['heading'])+','+str(prev_row['heading'])
            merged=list(set(merged.split(',')))
            # row['heading']=','.join(list(set([(row['heading']+','+prev_row['heading'])].split(','))))

        else:
            print('reached')
            temp+=1
            
            prev_row=rows[i-temp] if i>temp-1 else None
            # print(prev_row)
            # break
            next_row=rows[i+temp] if i<size-temp else None
            # print(prev_row)
            
    # break
            # temp+=1
            # if i>temp-1:
            #     prev_row=rows[i-temp]
            # else:
            #     prev_row=None
            # next_row=rows[i+temp] if i<size-temp else None

    i+=1

0
here
reached
here
reached
here
reached
here


TypeError: can only concatenate str (not "float") to str

In [109]:
columns = ['Unnamed: 0', 'chunkId', 'content', 'heading', 'originalLanguageChunk', 'StartPage', 'EndPage', 'originalLanguageHeading', 'ContentWordCount', 'pdfName', 'contentType', 'image', 'summary', 'status']

# Create DataFrame
new_df = pd.DataFrame(rows, columns=columns)
new_df = new_df[new_df['status']=='visited']
new_df=new_df.drop(columns=['Unnamed: 0','status'])

In [110]:
new_df=new_df.drop(columns=['Unnamed: 0','status'])

KeyError: "['Unnamed: 0', 'status'] not found in axis"

In [84]:
new_df.to_csv('output/english/FPO_Scheme_Guidelines_FINAL_English_splitted_merged.csv',index=False)

# Summary

In [ ]:
import openai
from dotenv import load_dotenv
import os

load_dotenv()
openai.api_key=os.environ.get("API_KEY")

def get_gpt_response(system_prompt,user_prompt):
  response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo-1106", temperature = 0,
                messages=[{"role": "system", "content": system_prompt},
                          {"role": "user", "content": user_prompt}
                ])
  gpt_response =  response["choices"][0]["message"]["content"]
  return(gpt_response)

In [ ]:

system_prompt = """You are an AI bot that summarises user provided content in 5 sentences.
The content that you are shared is part of the Samagra Processes and Policy Corpus. The data was extracted from the PDF and has two sections - Heading and Content.
These will be provided to you to summarize in the folloowing format

Heading: <Heading>
Content: <Content>

You dont say anything else apart from the 5 sentence summary. Do not reproduce either the Heading or Content.
"""
file='E:\PDF-Parser\output\english\FARMER PRODUCER ORGANISATIONS.csv'
df = pd.read_csv(file)

df['summary'] = ''
for i in range(df.shape[0]):
    print(i)
    if df.loc[i,'contentType']!='heading':
        content = "Heading:" + str(df.loc[i,'heading']) + "\n" + "Content: " + str(df.loc[i,'content'])
    
        df.loc[i,'summary'] =  get_gpt_response(system_prompt,content)
df.to_csv(file)

In [ ]:
# summary('E:\PDF-Parser\output\english\FARMER PRODUCER ORGANISATIONS.csv')
df.loc[0,'contentType']

In [ ]:
df

# Testing

In [129]:
import pandas as pd

In [146]:
df=pd.read_csv('output/english/FARMER PRODUCER ORGANISATIONS_new.csv')

In [147]:
df['len_heading'] = df['heading'].apply(lambda x: len(str(x).split()))

In [148]:
df.len_heading.describe()

count    398.000000
mean      19.716080
std       18.178857
min        1.000000
25%        9.000000
50%       15.000000
75%       22.000000
max      130.000000
Name: len_heading, dtype: float64

In [149]:
df[df.len_heading == 130]

,Unnamed: 0,chunkId,content,heading,originalLanguageChunk,StartPage,EndPage,originalLanguageHeading,ContentWordCount,pdfName,contentType,image,summary,len_heading
316,316,238,a. A duly registered FPC {(Farmer Producer Com...,",Attachment 3: Schemes Of Small Farmers Agribu...",NaN,NaN,NaN,NaN,181,input/md/english/FARMER PRODUCER ORGANISATIONS...,para,NaN,The Equity Grant Fund (EGF) Scheme for Farmer ...,130
317,317,238,It has a business plan and budget for next 18 ...,",Attachment 3: Schemes Of Small Farmers Agribu...",NaN,NaN,NaN,NaN,120,input/md/english/FARMER PRODUCER ORGANISATIONS...,para,NaN,The Equity Grant Fund (EGF) Scheme for Farmer ...,130


# Combining MD

In [104]:
# List of input MD files
import os
destination='input/md/hindi/spa/'
md_files=files=os.listdir(destination)

print(md_files)



# Output combined file
output_file = 'SAP2023_24.md'

# Combine content of MD files
combined_content = ''

for md_file in md_files:
    md_file=f'{destination}{md_file}'
    print(md_file)
    with open(md_file, 'r', encoding='utf-8') as file:
        combined_content += file.read() + '\n\n'

# Write combined content to a new file
with open(output_file, 'w', encoding='utf-8') as combined_file:
    combined_file.write(combined_content)

print(f"Content of {len(md_files)} MD files combined and saved to {output_file}.")


['SAP2023_24_output_chunk_1.md', 'SAP2023_24_output_chunk_2.md']
input/md/hindi/spa/SAP2023_24_output_chunk_1.md
input/md/hindi/spa/SAP2023_24_output_chunk_2.md
Content of 2 MD files combined and saved to SAP2023_24.md.


In [101]:
md_files = [f for f in os.listdir(destination) if os.path.isfile(os.path.join(destination, f))]

In [102]:
df['content'] = df.apply(lambda row: translate_text_bhashini(row['originalLanguageChunk'])[0] , axis=1)

['ManjushaBookPart_I_output_chunk_1.md',
 'ManjushaBookPart_I_output_chunk_2.md',
 'ManjushaBookPart_I_output_chunk_3.md',
 'ManjushaBookPart_I_output_chunk_4.md']

# params

- scanned or unscanned
- english or hindi or oriya

# DUMP

In [ ]:
combined_chunks = []
inside_table = False
current_table = ''

for chunk in chunks:
    if inside_table:
        # Check if the current chunk is the end of the table
        if chunk.startswith('|'):
            inside_table = True
            current_table=current_table+'\n'+chunk
        else:
            combined_chunks.append(current_table)
            combined_chunks.append(chunk)
            inside_table=False
    elif chunk.startswith('|'):
        inside_table = True
        current_table=chunk
#     elif chunk=='':
#         continue
    else:
        combined_chunks.append(chunk)

In [ ]:
inside_pair=False
current_pair=''
for chunk in combined_chunks:
    if chunk['type']=='heading':
        iside_pair=True
    elif chunk['type']=='para':
        if inside_pair

In [ ]:
for i in combined_chunks:
    print(i,end='\n************\n')

## GPT Stuff (No use)

In [ ]:
from markdown_it import MarkdownIt
from bs4 import BeautifulSoup

In [ ]:
file='Value_chain_financing.md'
file=open(file,'r',encoding='utf-8')
md_text=file.read()

In [ ]:
from markdown_it import MarkdownIt

def md_to_html(md_file_path, html_file_path):
    # Read Markdown content from file
    with open(md_file_path, 'r', encoding='utf-8') as md_file:
        md_text = md_file.read()
    md = MarkdownIt()
    html_content = md.render(md_text)
    with open(html_file_path, 'w', encoding='utf-8') as html_file:
        html_file.write(html_content)

md_path = "Value_chain_financing.md"
html_path = "output.html"
md_to_html(md_path, html_path)


In [ ]:
file=open('output.html', 'r', encoding='utf-8')
html_content = file.read()
page_data = {}
current_main_heading = ''
current_subheading = ''
crop_soup = BeautifulSoup(html_content, 'html.parser')
# print(crop_soup)
tags = crop_soup.find_all(['h4','h1','h2','h3', 'p', 'li','table'])

for tag in tags:
    if tag.name == 'h4':
        current_main_heading = tag.text.strip()
        current_subheading = ''  # Reset subheading when a new main heading is found
        if current_main_heading not in page_data:
            page_data[current_main_heading] = {}
    elif tag.name == 'p' and tag.find('strong'):
        current_subheading = tag.text.strip()
        if current_main_heading not in page_data:
            page_data[current_main_heading] = {}
        if current_subheading not in page_data[current_main_heading]:
            page_data[current_main_heading][current_subheading] = []
    elif current_main_heading and current_subheading:
        # Only add content if both main heading and subheading are available
        if tag.name=='table':
            data=table_parser(tag)
        else:
            data=tag.text.strip()
        page_data[current_main_heading][current_subheading].append(data)


    else:
        if current_main_heading and not current_subheading:
            current_subheading='General'
            page_data[current_main_heading][current_subheading]=[]
        if not current_main_heading and not current_subheading:
            current_main_heading='General'
            current_subheading='General'
            page_data[current_main_heading]={}
            page_data[current_main_heading][current_subheading]=[]
        if tag.name=='table':
            data=table_parser(tag)
        else:
            data=tag.text.strip()
        page_data[current_main_heading][current_subheading].append(data)


In [ ]:
page_data

In [ ]:
md = MarkdownIt()
tokens = md.parse(md_text)

headers = []
current_header = None

for token in tokens:
    print(token,end='\n\n\n')